<a href="https://colab.research.google.com/github/aquapathos/FooocusJAni/blob/main/Fooocus_onColabwithMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://i.imgur.com/WaGbhNe.png)


----
- とにかく試したいなら「ランタイム」メニューで「すべてのセルを実行」を選び、２の下部の「<font color='darkred'>**約５分かかります**</font>」に進んでください。
- 無料ユーザの場合、GPUが割り当てられないことがあります。「<font color='darkblue'>**GPUバックエンドに接続できません**</font>」と表示された場合は、GPUが使えません。その場合、制限はありますが、CPUだけで動作するようにしてあります。
- CPUだけの場合、相当時間がかかります（体感で10倍ぐらい）。メモリも不足するため、速度重視で質を落とす設定となっております。本来の画像品質は得られなことをご承知ください。
----
# Foooocus + MM LoRA

lllyasvielさんの作られた [Fooocus Fooocus](https://github.com/lllyasviel/Fooocus) に

- 日本語メニューのリソース
- 画家　[渡辺美香子さんの色鉛筆画](https://www.facebook.com/mikakostudio/)風LoRA　のプリセット  （絵の例は[こちら](https://wgarts.stores.jp/)）
- [Fooocus_Extensions_AlekPet](https://github.com/AlekPet/Fooocus_Extensions_AlekPet)

を加えて起動できるようにした Google Colaboratory のノート（pythonプログラム)を作りました。  
日本語化しての起動については、[Fooocus 日本語アニメ特化版 j.aicu.ai/FoooC](https://j.aicu.ai/FoooC)を参考にさせてもらっています。
さらに、[Fooocus_Extensions_AlekPet](https://github.com/AlekPet/)のプロンプト翻訳機能も使えるようにしてあります。


## 変更点
1. pygit2==1.12.2　→1.15.1
2. アニメ用のモデルを、animaPencilXL_v310.safetensors から　animagine-xl-3.0.safetensors　に変更
3. MM(Mikako Method)のプリセットを追加
4. 日本語化リソースの強化
5. CPU Only対応　（ただし10倍ほど時間がかかるので実用性は低い）

# １．画像出力フォルダの設定

- 出力画像をGoogle Driveに残すことができます。  
残したい場合は下のセルで「Google Driveに保存」にチェックを入れ、パスをセットしてください。
> - /content/gdrive/MyDrive/がGoogle Drive 直下を意味します。  
> - ローカルなストレージ（例えばCドライブ）は指定できません。  
> - 出力画像をGoogle ドライブに残す場合は、アクセスを許可を求めるダイアログが開きますので、許可し「Googleドライブに接続」を完了してください。
- Google Driveに残さない場合でも、セッションが生きている間は画像を参照可能です。「高度な設定」にチェックを入れ、Setting タブの一番下のHistory Logからアクセスできます。

### 参考
- 生成画像は１枚２MB前後、約500枚で１GB
- Google ドライブの無料ユーザーの容量は15 GB


### <font color=red>↓実行してください</font>


In [ ]:
# GPU のチェック
import torch
# Check for GPU availability
GPU_ready = torch.cuda.is_available()
if GPU_ready:
  print("GPU が使えます")
else:
  print("GPU が割り当てられていないため CPUのみで実行します")
  print("非常に時間がかかりますがご了承ください")

# @markdown #保存フォルダの選択
from google.colab import drive
import os

# オプションの設定

Googleドライブに保存 = False  # @param {type:"boolean"}

if Googleドライブに保存:
  # Google Driveをマウント
  drive.mount('/content/gdrive')
  保存フォルダ = "/content/gdrive/MyDrive/Fooocus-outputs"  # @param {type:"string"}

target_local_dir = "/content/Fooocus/outputs"

%cd /content
try:
  setup_finished
except NameError:
  # 必要なパッケージのインストール
  !pip -qq install pygit2==1.15.1
  !pip -qq install torchsde==0.2.6
  !rm -fr Fooocus
  !git clone https://github.com/lllyasviel/Fooocus.git

  if Googleドライブに保存:
    # シンボリックリンクの作成
    if os.path.exists(保存フォルダ ):
      if os.path.islink(保存フォルダ ):
        os.unlink(保存フォルダ )
      os.symlink(保存フォルダ,target_local_dir)
    else:
      # Google 出力ディレクトリを作成
      os.makedirs(target_local_dir, exist_ok=True)

# 日本語パッチを追加
ja_json_url = "https://raw.githubusercontent.com/aquapathos/FooocusJAni/refs/heads/main/ja.json"
ja_json_path = "/content/Fooocus/language/ja.json"
os.makedirs(os.path.dirname(ja_json_path), exist_ok=True)
!curl -L -o {ja_json_path} {ja_json_url}
print(f"ja.json downloaded and saved to {ja_json_path}")

# MM用のプリセットを追加
mm_json_url = "https://huggingface.co/iciromaco/mm/raw/main/mm_preset.json"
mm_json_path = "/content/Fooocus/presets/mm_preset.json" if GPU_ready else "/content/Fooocus/presets/mm_cpu_preset.json"
os.makedirs(os.path.dirname(mm_json_path), exist_ok=True)
!curl -L -o {mm_json_path} {mm_json_url}
print(f"mm_preset.json downloaded and saved to {mm_json_path}")

# アニメ版の設定ファイルのパッチ(最新版)を追加
anime_json_url = "https://raw.githubusercontent.com/aquapathos/FooocusJAni/refs/heads/main/anime.json"
anime_json_path = "/content/Fooocus/presets/anime.json"
os.makedirs(os.path.dirname(anime_json_path), exist_ok=True)
!curl -L -o {anime_json_path} {anime_json_url}
print(f"anime.json downloaded and saved to {anime_json_path}")

setup_finished = True


## ２．ウェブアプリの起動

#### モデルタイプ： 次の３つから選んでください
- デフォルト : Fooocus のデフォルト設定
- アニメ：　アニメ画生成
- MM：Mikako Methodの風景画生成

モデルのダウンロードに少し時間がかかります。しばらくお待ちください。

### 日本語メニュー，翻訳機能の利用の有無，モデルタイプを選び<font color=red>↓実行してください</font>

In [ ]:
%cd /content/Fooocus

# オプションの設定
日本語メニュー = True  # @param {type:"boolean"}
翻訳機能 = True  # @param {type:"boolean"}
モデルタイプ = "MM"  # @param ["デフォルト", "アニメ","MM"] {allow-input: true}

if 翻訳機能:
  print("H")
  !git clone https://github.com/AlekPet/Fooocus_Extensions_AlekPet.git
  !mv Fooocus_Extensions_AlekPet/prompt_translate/webui.py .
  !mv Fooocus_Extensions_AlekPet/prompt_translate/modules/* ./modules
  !sed -i.bak "s/label='Enable translate', value=False/label='Enable translate', value=True/" modules/module_translate.py
  !sed -i.bak 's/"default_prompt": "[^"]*"/"default_prompt": "湖に映る逆さ絵の風景、人はいない、小さな田舎の家が建っている、家の前には赤いレトロな車、手前には色とりどりのコスモス畑"/' presets/mm_preset.json
  if not GPU_ready:
    !sed -i.bak 's/"default_prompt": "[^"]*"/"default_prompt": "湖に映る逆さ絵の風景、人はいない、小さな田舎の家が建っている、家の前には赤いレトロな車、手前には色とりどりのコスモス畑"/' presets/mm_cpu_preset.json
    !sed -i.bak 's/HYPER_SD = 4/HYPER_SD = 3/' modules/flags.py
    !sed -i.bak 's/1664\*576/832\*576/' modules/flags.py

if GPU_ready:
  # GPUが使える場合
  if モデルタイプ == "デフォルト" and not 日本語メニュー:
      !python entry_with_update.py  --always-high-vram  --output-path {target_local_dir} --share
  elif モデルタイプ == "デフォルト" and 日本語メニュー :
      !python entry_with_update.py  --always-high-vram  --output-path {target_local_dir} --share --language ja
  elif モデルタイプ == "アニメ" and not 日本語メニュー:
      !python entry_with_update.py --preset anime --always-high-vram --output-path {target_local_dir} --share
  elif モデルタイプ == "アニメ" and 日本語メニュー:
      !python entry_with_update.py --preset anime --always-high-vram --output-path {target_local_dir} --share --language ja
  elif モデルタイプ == "MM" and not 日本語メニュー:
      !python entry_with_update.py --preset mm_preset --always-high-vram --output-path {target_local_dir} --share
  elif モデルタイプ == "MM" and 日本語メニュー:
      !python entry_with_update.py --preset mm_preset --always-high-vram --output-path {target_local_dir} --share --language ja
else:
  # CPU Only
  if モデルタイプ == "デフォルト" and not 日本語メニュー:
      !python entry_with_update.py --always-cpu 1 --vae-in-fp16 --unet-in-fp8-e5m2 --all-in-fp16 --attention-split --output-path {target_local_dir} --share
  elif モデルタイプ == "デフォルト" and 日本語メニュー :
      !python entry_with_update.py --always-cpu 1 --vae-in-fp16 --unet-in-fp8-e5m2 --all-in-fp16 --attention-split --output-path {target_local_dir} --share --language ja
  elif モデルタイプ == "アニメ" and not 日本語メニュー:
      !python entry_with_update.py  --always-cpu 1 --vae-in-fp16 --unet-in-fp8-e5m2 --all-in-fp16 --attention-split --preset anime_cpu --output-path {target_local_dir} --share
  elif モデルタイプ == "アニメ" and 日本語メニュー:
      !python entry_with_update.py  --always-cpu 1 --vae-in-fp16 --unet-in-fp8-e5m2 --all-in-fp16 --attention-split --preset anime_cpu --output-path {target_local_dir} --share --language ja
  elif モデルタイプ == "MM" and not 日本語メニュー:
      !python entry_with_update.py --always-cpu 1 --vae-in-fp16 --unet-in-fp8-e5m2 --all-in-fp16 --attention-split --preset mm_cpu_preset --always-high-vram --output-path {target_local_dir} --share
  elif モデルタイプ == "MM" and 日本語メニュー:
      !python entry_with_update.py --always-cpu 1 --vae-in-fp16 --unet-in-fp8-e5m2 --all-in-fp16 --attention-split --preset mm_cpu_preset --output-path {target_local_dir} --share --language ja


上欄に**gradio.live** で終わるURLが表示されるまでお待ちください（下図参照）。   
<font color='darkred'>**約５分かかります**</font>。  

URLが表示されたらクリックしてください。自動的にブラウザが開き、Fooocusアプリが表示されます。

<img src="https://i.imgur.com/YARA2nf.png" width = 500></img>

## 翻訳機能
アプリが立ち上がったあと、アプリ上で「**翻訳機能　有効**」にチェックをいれておけば，日本語でプロンプトを入力することができます。（翻訳機能はデフォルトでオン，日本語以外でもOK）

<img src="https://i.imgur.com/yAeIJyQ.png" width = 500>></img>



|||
| ---- | ---- |
|![](https://i.imgur.com/jaQBei0.png)|![](https://i.imgur.com/XwvS5Zx.png)|

----
>#### 左
>upside-down landscape reflected in lake, no human, a small country house stands, a red roadster in front of the house, a colorful cosmos field in the foreground

>#### 右
>flower, outdoors, sky, tree, no humans, night, traditional media, grass, star sky, night sky, scenery, starry sky, mountain, painting medium, field, house




## 参考URL
- [Fooocusのスタイル一覧](https://note.com/disc225/n/n2599cab73410)
- [Fooocus_Style_Reference](https://docs.google.com/spreadsheets/d/1AF5bd-fALxlu0lguZQiQVn1yZwxUiBJGyh2eyJJWl74/edit?gid=0#gid=0)
- [fooocusの使い方](https://poseroboegaki.com/ai-illust/)